## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import dataframe_image as dfi

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/bweir/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,69,20,9.22,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.25,60,0,7.65,clear sky
2,2,Puerto Ayora,EC,-0.7393,-90.3518,77.00,92,16,1.01,few clouds
3,3,Deputatskiy,RU,69.3000,139.9000,-1.57,90,82,2.89,broken clouds
4,4,Ancud,CL,-41.8697,-73.8203,54.00,88,100,11.01,overcast clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like? '))
max_temp = float(input('What is the maximum temperature you would like? '))

What is the minimum temperature you would like? 60
What is the maximum temperature you would like? 85


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp']<= max_temp)]
city_filtered_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,69,20,9.22,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.25,60,0,7.65,clear sky
2,2,Puerto Ayora,EC,-0.7393,-90.3518,77.00,92,16,1.01,few clouds
5,5,Montoro,ES,38.0241,-4.3834,60.80,72,0,3.44,clear sky
6,6,Bambous Virieux,MU,-20.3428,57.7575,80.01,88,75,9.22,light rain


In [40]:
# 4a. Determine if there are any empty rows.
city_filtered_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_filtered_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except:
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name
empty_hotel = hotel_df[hotel_df['Hotel Name'] == ''].index
clean_hotel_df = hotel_df.drop(empty_hotel)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Rikitea,PF,77.25,clear sky,-23.1203,-134.9692,Pension Maro'i
2,Puerto Ayora,EC,77.00,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Montoro,ES,60.80,clear sky,38.0241,-4.3834,Hotel Mirador De Montoro
6,Bambous Virieux,MU,80.01,light rain,-20.3428,57.7575,Casa Tia Villa


In [46]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \N{DEGREE SIGN}F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info) 
# 11b. Display the figure
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))